In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import networkx as nx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

# Summarizing Text

Let's try out extractive summarization using the first four paragraphs of [The Great Gatsby](http://gutenberg.net.au/ebooks02/0200041h.html).

First, we'll try to extract the most representative sentence.  Then, we'll extract keywords.

## Sentence extraction

The steps of our sentence extraction process:

1. Parse and tokenize the text using spaCy, and divide into sentences.
2. Calculate the tf-idf matrix.
3. Calculate similarity scores.
4. Calculate TextRank: We're going to use the ´networkx´ package to run the TextRank algorithm.

Let's get started!


In [2]:
# Importing the text the lazy way.
gatsby='''In my younger and more vulnerable years my father gave me some advice that I've been turning over in 
my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all 
the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've 
always been unusually communicative in a reserved way, and I understood that he meant a great deal more 
than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious 
natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect 
and attach itself to this quality when it appears in a normal person, and so it came about that in college 
I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown 
men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile 
levity when I realized by some unmistakable sign that an intimate revelation was quivering on the 
horizon--for the intimate revelations of young men or at least the terms in which they express them are 
usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. 
I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and 
I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after 
boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on 
the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came 
back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral 
attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only 
Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented 
everything for which I have an unaffected scorn. If personality is an unbroken series of successful 
gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of 
life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles
away. This responsiveness had nothing to do with that flabby impressionability which is dignified under th
name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I
have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned 
out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams 
that temporarily closed out my interest in the abortive sorrows and short-winded elations of men.
'''

# We want to use the standard english-language parser.
parser = spacy.load('en')

# Parsing Gatsby.
gatsby = parser(gatsby)

In [3]:
gatsby[:100]

In my younger and more vulnerable years my father gave me some advice that I've been turning over in 
my mind ever since. "Whenever you feel like criticizing any one," he told me, "just remember that all 
the people in this world haven't had the advantages that you've had." He didn't say any more but we've 
always been unusually communicative in a reserved way, and I understood that he meant a great deal more 
than that. In consequence I'm inclined

In [4]:
# Dividing the text into sentences and storing them as a list of strings.
sentences=[]
for span in gatsby.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)

# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=False, 
                          stop_words=None,
                          ngram_range=(1, 1), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)
print(data_counts)

  (0, 215)	0.23363826510307834
  (0, 74)	0.20400876547714022
  (0, 154)	0.20400876547714022
  (0, 122)	0.12372730168356723
  (0, 173)	0.23363826510307834
  (0, 245)	0.23363826510307834
  (0, 42)	0.20400876547714022
  (0, 259)	0.18298630093544344
  (0, 21)	0.23363826510307834
  (0, 219)	0.18298630093544344
  (0, 150)	0.18298630093544344
  (0, 97)	0.23363826510307834
  (0, 80)	0.20400876547714022
  (0, 278)	0.23363826510307834
  (0, 262)	0.23363826510307834
  (0, 157)	0.16668001060865378
  (0, 281)	0.23363826510307834
  (0, 158)	0.46007040392851606
  (0, 6)	0.20400876547714022
  (1, 20)	0.21114195739978456
  (1, 106)	0.36873078175403246
  (1, 110)	0.21114195739978456
  (1, 277)	0.18436539087701623
  (1, 238)	0.15063090664451403
  (1, 175)	0.21114195739978456
  :	:
  (16, 125)	0.17510391992666682
  (16, 53)	0.17510391992666682
  (16, 229)	0.17510391992666682
  (16, 69)	0.17510391992666682
  (16, 263)	0.17510391992666682
  (16, 87)	0.17510391992666682
  (16, 70)	0.17510391992666682
  (16, 

# Similarity

So far, this is all (hopefully) familiar: We've done text parsing and the tf-idf calculation before.  We should now have sentences represented as vectors, with each word having a score based on how often it occurs in the sentence divided by how often it occurs in the whole text.

Now let's calculate the similarity scores for the sentences and apply the TextRank algorithm.  Because TextRank is based on Google's PageRank algorithm, the function is called 'pagerank'.  The hyperparameters are the damping parameter ´alpha´ and the convergence parameter ´tol´.

In [5]:
# Calculating similarity
similarity = data_counts * data_counts.T

# Identifying the sentence with the highest rank.
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks = nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)),
                reverse=True)
print(ranked[0])


(0.07038127733413584, 'This responsiveness had nothing to do with that flabby impressionability which is dignified under th\nname of the "creative temperament"--it was an extraordinary gift for hope, a romantic readiness such as I\nhave never found in any other person and which it is not likely I shall ever find again.')


Since a lot of Gatsby is about the narrator acting as the observer of other peoples' sordid secrets, this seems pretty good.  Now, let's extract some keywords.

# Keyword summarization

1) Parse and tokenize text (already done).  
2) Filter out stopwords, choose only nouns and adjectives.  
3) Calculate the neighbors of words (we'll use a window of 4).  
4) Run TextRank on the neighbor matrix.  


In [6]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_filt = [word for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
words=set(gatsby_filt)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(gatsby):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in gatsby_filt]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')
        



done!


In [7]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])


[(0.014171122776786298, hope), (0.012835923559225273, promises), (0.012835923559225273, exempt), (0.012750774149125241, glimpses), (0.012491464113668544, intimate)]


/home/uzi/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


These results are less impressive.  'Hope', 'promises', and 'glimpses' certainly fit the elegiac, on-the-outside-looking-in tone of the book, but 'exempt' and 'world' (or 'intimate' in our case) are pretty generic.  TextRank may perform better on a larger text sample.

# Drill

It is also possible that keyword phrases will work better.  Modifiy the keyword extraction code to extract two-word phrases (digrams) rather than single words.  Then try it with trigrams.  You will probably want to broaden the window that defines 'neighbors.'  Try a few different modifications, and write up your observations in your notebook.  Discuss with your mentor.

In [13]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_filt = [word for word in gatsby if word.is_stop==False and word.is_punct== False and (word.pos_=='NOUN' or word.pos_=='ADJ')]

two_grams = []
for idx, word in enumerate(gatsby_filt):
    two_gram = ''.join(gatsby[i].string for i in range(idx, idx + 2)).strip()
    two_grams.append(two_gram)
                     
two_phrase = set(two_grams)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=two_phrase,index=two_phrase,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(gatsby):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in gatsby_filt]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

KeyError: younger

In [14]:
two_grams

['In my',
 'my younger',
 'younger and',
 'and more',
 'more vulnerable',
 'vulnerable years',
 'years my',
 'my father',
 'father gave',
 'gave me',
 'me some',
 'some advice',
 'advice that',
 'that I',
 "I've",
 "'ve been",
 'been turning',
 'turning over',
 'over in',
 'in',
 'my',
 'my mind',
 'mind ever',
 'ever since',
 'since.',
 '. "',
 '"Whenever',
 'Whenever you',
 'you feel',
 'feel like',
 'like criticizing',
 'criticizing any',
 'any one',
 'one,',
 ',"',
 '" he',
 'he told',
 'told me',
 'me,',
 ', "',
 '"just',
 'just remember',
 'remember that',
 'that all',
 'all',
 'the',
 'the people',
 'people in',
 'in this',
 'this world',
 'world have',
 "haven't",
 "n't had",
 'had the',
 'the advantages',
 'advantages that',
 'that you',
 "you've",
 "'ve had",
 'had.',
 '."',
 '" He',
 'He did',
 "didn't",
 "n't say",
 'say any',
 'any more',
 'more but',
 'but we',
 "we've",
 "'ve",
 'always',
 'always been',
 'been unusually',
 'unusually communicative',
 'communicative in',

In [15]:
gatsby_filt[1].string.strip()

'vulnerable'

In [16]:
two_phrase

{'" He',
 '" he',
 '"Whenever',
 '"just',
 "'m inclined",
 "'ve",
 "'ve been",
 "'ve had",
 ', "',
 ', a',
 ', and',
 ',"',
 '. "',
 '. In',
 '."',
 'He did',
 'I understood',
 "I'm",
 "I've",
 'In consequence',
 'In my',
 'Whenever you',
 'a few',
 'a great',
 'a habit',
 'a reserved',
 'advantages that',
 'advice that',
 'all',
 'all judgments',
 'also made',
 'always',
 'always been',
 'and I',
 'and also',
 'and more',
 'any more',
 'any one',
 'been turning',
 'been unusually',
 'bores.',
 'but we',
 'communicative in',
 'consequence I',
 'criticizing any',
 'curious',
 'deal more',
 "didn't",
 'ever since',
 'father gave',
 'feel like',
 'few veteran',
 'gave me',
 'great deal',
 'habit that',
 'had the',
 'had.',
 'has opened',
 "haven't",
 'he meant',
 'he told',
 'in',
 'in a',
 'in this',
 'inclined to',
 'judgments,',
 'just remember',
 'like criticizing',
 'made me',
 'many curious',
 'me and',
 'me some',
 'me the',
 'me,',
 'meant a',
 'mind ever',
 'more',
 'more but',
 

In [17]:
adjacency

,few veteran,"way,",me some,". """,always,that has,opened up,habit that,"me,","""just",...,that I,"""Whenever",not a,vulnerable years,", and",we've,in,more,In consequence,in a
few veteran,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"way,",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
me some,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
". """,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
always,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
that has,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
opened up,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
habit that,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"me,",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""just",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
from tqdm import tqdm # show progress bar
import time # time random searches

In [83]:
########################################################################
#########    Creating a function in order to quickly call upon    ######
#########         the model with different features  while        ######
#########        cycling through different model parameters       ######
########################################################################

def Counter(sentences, ngram_range=[(1, 1), (1,2), (1,3)],
                       max_df=[.5, .75, 1.0] 
            ):
    
    tfidf_hyperparams = dict(
                             ngram_range=ngram_range, 
                             max_df=max_df, 
                            )

    tfidf_hyper_table = list()

    # Timed for loop...start the clock!
    start_time = time.time()
    for trial in tqdm(range(10), position=0):
        params = dict()
        for k, v in tfidf_hyperparams.items():
            i = np.random.randint(len(v))
            params[k] = v[i]
            
            
            
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_filt = [word for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
words=set(gatsby_filt)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(gatsby):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in gatsby_filt]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1
        
        #print(params)
        params['ranked words'] = ranked[:5]

        # Store our parameters and score in a dataframe
        tfidf_hyper_table.append(params)

        end_time = time.time()

    print(f'Execution time: {round(end_time - start_time, 2)} seconds')
    tfidf_df = pd.DataFrame(data=tfidf_hyper_table)
    return tfidf_df#.sort_values(by='cross_val_mean', ascending=False)

In [84]:
Counter(sentences)

 70%|███████   | 7/10 [00:00<00:00, 26.76it/s]

[(0.06711863568669645, 'The abnormal mind is quick to detect \nand attach itself to this quality when it appears in a normal person, and so it came about that in college \nI was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown \nmen.'), (0.06463710330278168, 'No--Gatsby turned \nout all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams \nthat temporarily closed out my interest in the abortive sorrows and short-winded elations of men.'), (0.06425754510044927, 'This responsiveness had nothing to do with that flabby impressionability which is dignified under th\nname of the "creative temperament"--it was an extraordinary gift for hope, a romantic readiness such as I\nhave never found in any other person and which it is not likely I shall ever find again.'), (0.06240998815751511, 'And, after \nboasting this way of my tolerance, I come to the admission that it has a limit.'), (0.06096922851731912,

100%|██████████| 10/10 [00:00<00:00, 28.96it/s]

[(0.06396054146379329, 'The abnormal mind is quick to detect \nand attach itself to this quality when it appears in a normal person, and so it came about that in college \nI was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown \nmen.'), (0.06279584362278391, 'This responsiveness had nothing to do with that flabby impressionability which is dignified under th\nname of the "creative temperament"--it was an extraordinary gift for hope, a romantic readiness such as I\nhave never found in any other person and which it is not likely I shall ever find again.'), (0.0615043133968137, 'No--Gatsby turned \nout all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams \nthat temporarily closed out my interest in the abortive sorrows and short-winded elations of men.'), (0.060580770052749, 'And, after \nboasting this way of my tolerance, I come to the admission that it has a limit.'), (0.06018451548843599, "I

,ngram_range,max_df,ranked words
0,"(1, 2)",1.00,"[(0.06711863568669645, The abnormal mind is qu..."
1,"(1, 3)",0.50,"[(0.06278290784595786, This responsiveness had..."
2,"(1, 3)",0.50,"[(0.06278290784595786, This responsiveness had..."
3,"(1, 1)",0.75,"[(0.07241318291926943, The abnormal mind is qu..."
4,"(1, 3)",0.75,"[(0.06396054146379329, The abnormal mind is qu..."
5,"(1, 1)",0.50,"[(0.07073858537199645, This responsiveness had..."
6,"(1, 2)",1.00,"[(0.06711863568669645, The abnormal mind is qu..."
7,"(1, 3)",0.75,"[(0.06396054146379329, The abnormal mind is qu..."
8,"(1, 3)",1.00,"[(0.06431134900595412, The abnormal mind is qu..."
9,"(1, 3)",1.00,"[(0.06431134900595412, The abnormal mind is qu..."
